In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
from pylab import rcParams
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing

import tensorflow as tf
from keras import optimizers, Sequential
from keras.models import Model
from keras.utils import plot_model
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten
from keras.callbacks import History
from keras import callbacks

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import recall_score, classification_report, auc, roc_curve
from sklearn.metrics import precision_recall_fscore_support, f1_score

from wtte.wtte import WeightWatcher
import wtte.wtte as wtte
import wtte.weibull as weibull

from numpy.random import seed
seed(7)
from tensorflow import set_random_seed
set_random_seed(11)

from sklearn.model_selection import train_test_split

SEED = 123 #used to help randomly select the data points
DATA_SPLIT_PCT = 0.2
np.random.seed(2)
pd.set_option("display.max_rows", 1000)

In [ ]:
id_col = 'id'
time_col = 'cycle'
feature_cols = ['setting1', 'setting2', 'setting3', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21']
column_names = [id_col, time_col] + feature_cols

np.set_printoptions(suppress=True, threshold=10000)
train_orig = pd.read_csv('train_FD001.txt', sep=" ", header=None)
train_orig.drop(train_orig.columns[[26, 27]], axis=1, inplace=True)
train_orig.columns = column_names
test_x_orig = pd.read_csv('test_FD001.txt', sep=" ", header=None)
test_x_orig.drop(test_x_orig.columns[[26, 27]], axis=1, inplace=True)
test_x_orig.columns = column_names
test_y_orig = pd.read_csv('RUL_FD001.txt', header=None, names=['T'])
test_y_orig.columns=['rul']

In [ ]:
test_x_orig.shape

In [ ]:
test_x_orig.set_index(['id', 'cycle'], verify_integrity=True)
train_orig.set_index(['id', 'cycle'], verify_integrity=True)

In [ ]:
#minmaxScaler
train_orig['cycle_norm'] = train_orig['cycle']
cols_normalize = train_orig.columns.difference(['id','cycle','RUL'])
min_max_scaler = preprocessing.MinMaxScaler()
norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(train_orig[cols_normalize]), 
                             columns=cols_normalize, 
                             index=train_orig.index)
join_df = train_orig[train_orig.columns.difference(cols_normalize)].join(norm_train_df)
train_orig = join_df.reindex(columns = train_orig.columns)
train_orig.head()

In [ ]:
test_x_orig['cycle_norm'] = test_x_orig['cycle']
norm_test_df = pd.DataFrame(min_max_scaler.fit_transform(test_x_orig[cols_normalize]), 
                            columns=cols_normalize, 
                            index=test_x_orig.index)
test_join_df = test_x_orig[test_x_orig.columns.difference(cols_normalize)].join(norm_test_df)
test_x_orig = test_join_df.reindex(columns = test_x_orig.columns)
test_x_orig = test_x_orig.reset_index(drop=True)
test_x_orig.head()

In [ ]:
from sklearn import pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Combine the X values to normalize them, 
all_data_orig = pd.concat([train_orig, test_x_orig])
# all_data = all_data[feature_cols]
# all_data[feature_cols] = normalize(all_data[feature_cols].values)
feature_cols = ['setting1', 'setting2', 'setting3', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21']

scaler=pipeline.Pipeline(steps=[
#     ('z-scale', StandardScaler()),
     ('minmax', MinMaxScaler(feature_range=(-1, 1))),
     ('remove_constant', VarianceThreshold())
])

all_data = all_data_orig.copy()
test_data = test_x_orig[test_x_orig['id'] == 13]

all_data = np.concatenate([all_data_orig[['id', 'cycle']], scaler.fit_transform(all_data[feature_cols])], axis=1)
test_data = np.concatenate([test_data[['id', 'cycle']], scaler.fit_transform(test_data[feature_cols])], axis=1)
print(test_data.shape)
print(type(test_data))


In [ ]:
# then split them back out
train = all_data[0:train_orig.shape[0], :]
test = all_data[train_orig.shape[0]:, :]

# Make engine numbers and days zero-indexed, for everybody's sanity
train[:, 0:2] -= 1
test[:, 0:2] -= 1
test_data[:,0:2] -= 1

test_data.shape
id_df = pd.DataFrame(train, columns = ['id', 'cycle'] + feature_cols[0:17])
id_df['cycle'].shape

id_test_df = pd.DataFrame(test, columns = ['id', 'cycle'] + feature_cols[0:17])
id_test_df['cycle'].shape


In [ ]:
import tqdm
from tqdm import tqdm

# TODO: replace using wtte data pipeline routine
def build_data(engine, time, x, max_time, is_test, mask_value, is_test_13):
    # y[0] will be days remaining, y[1] will be event indicator, always 1 for this data
    out_y = []
    
    # number of features
    d = x.shape[1]

    # A full history of sensor readings to date for each x
    out_x = []
    n_engines=100
    max_engine_time = 0
    
    if(is_test_13):
        for i in tqdm(range(n_engines)):
            if(i == 12):
                # When did the engine fail? (Last day + 1 for train data, irrelevant for test.)
                max_engine_time = int(np.max(time[engine == i])) + 1


                if is_test:
                    start = max_engine_time - 1
                else:
                    start = 0

                this_x = []

                for j in range(start, max_engine_time):
                    engine_x = x[engine == i]

                    out_y.append(np.array((max_engine_time - j, 1), ndmin=2))

                    xtemp = np.zeros((1, max_time, d))
                    xtemp += mask_value
        #             xtemp = np.full((1, max_time, d), mask_value)

                    xtemp[:, max_time-min(j, 99)-1:max_time, :] = engine_x[max(0, j-max_time+1):j+1, :]
                    this_x.append(xtemp)

                this_x = np.concatenate(this_x)
                out_x.append(this_x)
    else:
        for i in tqdm(range(n_engines)):
            # When did the engine fail? (Last day + 1 for train data, irrelevant for test.)
            max_engine_time = int(np.max(time[engine == i])) + 1


            if is_test:
                start = max_engine_time - 1
            else:
                start = 0

            this_x = []

            for j in range(start, max_engine_time):
                engine_x = x[engine == i]

                out_y.append(np.array((max_engine_time - j, 1), ndmin=2))

                xtemp = np.zeros((1, max_time, d))
                xtemp += mask_value
    #             xtemp = np.full((1, max_time, d), mask_value)

                xtemp[:, max_time-min(j, 99)-1:max_time, :] = engine_x[max(0, j-max_time+1):j+1, :]
                this_x.append(xtemp)

            this_x = np.concatenate(this_x)
            out_x.append(this_x)
    out_x = np.concatenate(out_x)
    out_y = np.concatenate(out_y)
    return out_x, out_y

In [ ]:
# # Configurable observation look-back period for each engine/day
max_time = 100
mask_value = -99

train_x, train_y = build_data(engine=train[:, 0], time=train[:, 1], x=train[:, 2:], max_time=max_time, is_test=False, mask_value=mask_value, is_test_13=False)
test_x,_ = build_data(engine=test[:, 0], time=test[:, 1], x=test[:, 2:], max_time=max_time, is_test=True, mask_value=mask_value, is_test_13=False)

test_new,_ = build_data(engine=test_data[:, 0], time=test_data[:, 1], x=test_data[:, 2:], max_time=max_time, is_test=True, mask_value=mask_value, is_test_13=True)

train_x_2, train_y_2 = build_data(engine=test[:, 0], time=test[:, 1], x=test[:, 2:], max_time=max_time, is_test=False, mask_value=mask_value, is_test_13=False)
train_x_2.shape

In [ ]:
test_y = test_y_orig.copy()
print('train_x', train_x.shape, 'train_y', train_y.shape, 'test_x', test_x.shape, 'test_y', test_y.shape)

In [ ]:
timesteps =  train_x.shape[1]
n_features =  train_x.shape[2]

epochs = 100
batch = 100
lr = 0.0001

In [ ]:
#reshape the data
train_x = train_x.reshape(train_x.shape[0], timesteps, n_features)
test_x = test_x.reshape(test_x.shape[0], timesteps, n_features)

In [ ]:
def flatten(X):
    '''
    Flatten a 3D array.
    
    Input
    X            A 3D array for lstm, where the array is sample x timesteps x features.
    
    Output
    flattened_X  A 2D array, sample x features.
    '''
    flattened_X = np.empty((X.shape[0], X.shape[2]))  # sample x features array.
    for i in range(X.shape[0]):
        flattened_X[i] = X[i, (X.shape[1]-1), :]
    return(flattened_X)

def scale(X, scaler):
    '''
    Scale 3D array.

    Inputs
    X            A 3D array for lstm, where the array is sample x timesteps x features.
    scaler       A scaler object, e.g., sklearn.preprocessing.StandardScaler, sklearn.preprocessing.normalize
    
    Output
    X            Scaled 3D array.
    '''
    for i in range(X.shape[0]):
        X[i, :, :] = scaler.transform(X[i, :, :])
        
    return X

In [ ]:
print(timesteps)
print(n_features)

In [ ]:
# Initialize a scaler using the training data.
scaler = StandardScaler().fit(flatten(train_x))
X_train_scaled = scale(train_x, scaler)
X_test_scaled = scale(test_x, scaler)

In [ ]:
import keras.backend as K
K.set_epsilon(1e-10)
print('epsilon', K.epsilon())


encoder_decoder = Sequential()
encoder_decoder.add(LSTM(10, activation='relu', input_shape=(timesteps, n_features), return_sequences=True))
encoder_decoder.add(LSTM(6, activation='relu', return_sequences=True))
encoder_decoder.add(LSTM(1, activation='relu'))
encoder_decoder.add(RepeatVector(timesteps))
encoder_decoder.add(LSTM(10, activation='relu', return_sequences=True))
encoder_decoder.add(LSTM(1, activation='relu', return_sequences=True))
encoder_decoder.add(TimeDistributed(Dense(17)))
encoder_decoder.summary()

adam = optimizers.Adam(lr)

loss = wtte.loss(kind='discrete',reduce_loss=False).loss_function
encoder_decoder.compile(loss="mse", optimizer=adam)

In [ ]:

# history = History()
# weightwatcher = WeightWatcher()
# nanterminator = callbacks.TerminateOnNaN()

# lstm_autoencoder.fit(train_x, train_x, 
#                     epochs=epochs, 
#                     batch_size=batch, 
#                     validation_data=(test_x, test_x),
#                     verbose=1,
#                     callbacks=[nanterminator,history,weightwatcher])

encoder_decoder_history = encoder_decoder.fit(X_train_scaled, X_train_scaled, 
                                              batch_size=batch, 
                                              epochs=epochs, 
                                              validation_data=(X_test_scaled, X_test_scaled),
                                              verbose=2).history

In [ ]:
# plt.plot(lstm_autoencoder_history['loss'], linewidth=2, label='Train')
# plt.plot(lstm_autoencoder_history['val_loss'], linewidth=2, label='Valid')
# plt.legend(loc='upper right')
# plt.title('Model loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.show()

rpt_vector_layer = Model(inputs=encoder_decoder.inputs, outputs=encoder_decoder.layers[3].output)
time_dist_layer = Model(inputs=encoder_decoder.inputs, outputs=encoder_decoder.layers[5].output)
encoder_decoder.layers

In [ ]:
rpt_vector_layer_output = rpt_vector_layer.predict(train_x[:1])
print('Repeat vector output shape', rpt_vector_layer_output.shape)
print('Repeat vector output sample')
print(rpt_vector_layer_output[0])

In [ ]:
time_dist_layer_output = time_dist_layer.predict(train_x[:1])
print('Time distributed output shape', time_dist_layer_output.shape)
print('Time distributed output sample')
print(time_dist_layer_output[0])

In [ ]:
encoder = Model(inputs=encoder_decoder.inputs, outputs=encoder_decoder.layers[2].output)

In [ ]:
train_encoded = encoder.predict(train_x)
train_encoded_2nd = encoder.predict(train_x)
# validation_encoded = encoder.predict(X_valid)
print('Encoded time-series shape', train_encoded.shape)
print('Encoded time-series sample', train_encoded[0])

for i in train_encoded:
    print(i)

In [ ]:
type(train_encoded)

In [ ]:
autoencoder = Model(inputs=encoder_decoder.inputs, outputs=encoder_decoder.layers[4].output)

In [ ]:
train_encoded = autoencoder.predict(train_x)
for i in train_encoded:
    print(i)

In [ ]:
test_encoded = encoder.predict(train_x_2)
test_encoded_2nd = encoder.predict(train_x_2)
test_encoded

In [ ]:
# train_encoded.shape
test_encoded.shape
df2 = pd.DataFrame(test_encoded, columns = ['s'])
df2.insert(0,'id',id_test_df['id'])
df2.insert(1,'cycle',id_test_df['cycle'])
# df2.insert(3, 's2', test_encoded_2nd)

df = pd.DataFrame(train_encoded, columns = ['s'])
df.insert(0,'id',id_df['id'])
df.insert(1,'cycle',id_df['cycle'])
# df['id'].append(id_df['id'])
# df['cycle'].append(id_df['cycle'])
# df.insert(3, 's2', train_encoded_2nd)
train_encoded_2 = df.to_numpy()
print(train_encoded_2.shape)

test_encoded_2 = df2.to_numpy()
# test_encoded_2.shape
df

In [ ]:
# # Configurable observation look-back period for each engine/day
max_time = 100
mask_value = -99

train_x, train_y = build_data(engine=train_encoded_2[:, 0], time=train_encoded_2[:, 1], x=train_encoded_2[:, 2:], max_time=max_time, is_test=False, mask_value=mask_value, is_test_13=False)
test_x,_ = build_data(engine=test_encoded_2[:, 0], time=test_encoded_2[:, 1], x=test_encoded_2[:, 2:], max_time=max_time, is_test=True, mask_value=mask_value, is_test_13=False)


In [ ]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

In [ ]:
tte_mean_train = np.nanmean(train_y[:,0])
mean_u = np.nanmean(train_y[:,1])

# Initialization value for alpha-bias 
init_alpha = -1.0/np.log(1.0-1.0/(tte_mean_train+1.0) )
init_alpha = init_alpha/mean_u
print('tte_mean_train', tte_mean_train, 'init_alpha: ',init_alpha,'mean uncensored train: ',mean_u)

In [ ]:
import keras.backend as K
import keras
from keras.layers import Masking
from keras.layers import LSTM, GRU
from keras.layers import Lambda

K.set_epsilon(1e-10)
print('epsilon', K.epsilon())

history = History()
weightwatcher = WeightWatcher()
nanterminator = callbacks.TerminateOnNaN()
# reduce_lr = callbacks.ReduceLROnPlateau(monitor='loss', 
#                                         factor=0.5, 
#                                         patience=50, 
#                                         verbose=0, 
#                                         mode='auto', 
#                                         epsilon=0.0001, 
#                                         cooldown=0, 
#                                         min_lr=1e-8)

n_features = train_x.shape[-1]
print(n_features)

# Start building our model
model = Sequential()

# Mask parts of the lookback period that are all zeros (i.e., unobserved) so they don't skew the model
model.add(Masking(mask_value=mask_value, input_shape=(None, 2)))

# model.add(BatchNormalization())

# LSTM is just a common type of RNN. You could also try anything else (e.g., GRU).
model.add(GRU(20, activation='tanh', recurrent_dropout=0.25))

# model.add(Dense(20))

# We need 2 neurons to output Alpha and Beta parameters for our Weibull distribution
model.add(Dense(2))

# Apply the custom activation function mentioned above
# model.add(Activation(activate))

model.add(Lambda(wtte.output_lambda, 
                 arguments={"init_alpha":init_alpha, 
                            "max_beta_value":100.0, 
                            "alpha_kernel_scalefactor":0.5
                           },
                ))

# Use the discrete log-likelihood for Weibull survival data as our loss function
loss = wtte.loss(kind='discrete',reduce_loss=False).loss_function

model.compile(loss=loss, optimizer=keras.optimizers.Adam(lr=.01, clipvalue=0.5))

In [ ]:
model.summary()

In [ ]:
try:
    model.fit(train_x, train_y,
          epochs=1,
          batch_size=100, 
          verbose=1,
          validation_data=(test_x, test_y),
          callbacks=[nanterminator,history,weightwatcher])
except:
    print("\nError - repeating")

    model.compile(loss=loss, optimizer=keras.optimizers.Adam(lr=.01, clipvalue=0.5))

    model.fit(train_x, train_y,
          epochs=1,
          batch_size=100, 
          verbose=1,
          validation_data=(test_x, test_y),
          callbacks=[nanterminator,history,weightwatcher])